# Fichier de nettoyage des données avant leur analyse

In [1]:
import os
import pandas as pd
import geopandas as gpd
gpd.options.use_pygeos = True

# Set the working directory
download_dir = "../../RAW"
prod_dir = "../../PROD"

/var/folders/pl/xywgn7qd4sv_s094vn5h3dtr0000gn/T/ipykernel_91391/767398250.py:4: UserWarning: pygeos support was removed in 1.0. geopandas.use_pygeos is a no-op and will be removed in geopandas 1.1.
  gpd.options.use_pygeos = True


## Nettoyage des données LCZ

In [2]:
# Import des données LCZ
subfold_lcz = "LCZ"

# La liste théorique des fichiers à télécharger
to_download = pd.read_csv(os.path.join(download_dir, subfold_lcz, "to_download.csv"))
to_download["title_clean"] = to_download["title"].str.replace(".zip", "")

# Ce qui est dans le dossier LCZ
actually_downloaded = os.listdir(os.path.join(download_dir, subfold_lcz))

# Ce qui est téléchargé
lcz_folders = [f for f in actually_downloaded if f in to_download["title_clean"].values]    

# Le fichier des communes
communes = gpd.read_file(os.path.join(prod_dir, "geo/admin", "communes_2024.shp"))
communes = communes[["label", "official_c", "geometry"]]


# On charge les données FILOSOFI
subfold_filosofi = "INSEE/FILOSOFI/2019/CARREAUX_200M"
filosofi = gpd.read_file(os.path.join(download_dir, subfold_filosofi, "carreaux_200m_met.shp"))



# On crée une fonction pour calculer les données pondérées
def weighted_filo(datbase, weight_col, values_col):
    copy = datbase.copy()
    for value_col in values_col:
        copy[value_col+"_w"] = copy[weight_col] * copy[value_col]
    return copy

os.makedirs(os.path.join(prod_dir, "geo/lcz/filo_com"), exist_ok=True)

In [7]:
# On créé un dataframe pour stocker les résultats par commune

summary_tot_lcz_filo_flat = pd.DataFrame(columns=["lcz",        "official_c", # CODE INSEE
                                                                    "surf_com_official", # Surface globale commune
                                                                    "surf_lcz_group_com", # Surface commune couverte par un groupe LCZ
                                                                    "surf_id_filo", # Surface LCZ-FILO dd'une catégorie LCZ
                                                                    "label", # Nom de la commune
                                                                    'ind_w', 'men_w',
       'men_pauv_w', 'men_1ind_w', 'men_5ind_w', 'men_prop_w', 'men_fmp_w',
       'ind_snv_w', 'men_surf_w', 'men_coll_w', 'men_mais_w', 'log_av45_w',
       'log_45_70_w', 'log_70_90_w', 'log_ap90_w', 'log_inc_w', 'log_soc_w',
       'ind_0_3_w', 'ind_4_5_w', 'ind_6_10_w', 'ind_11_17_w', 'ind_18_24_w',
       'ind_25_39_w', 'ind_40_54_w', 'ind_55_64_w', 'ind_65_79_w', 'ind_80p_w',
       'ind_inc_w'])


summary_tot_lcz_filo_psdovol = pd.DataFrame(columns=["lcz", "official_c", # CODE INSEE
                                                                    "surf_com_official", # Surface globale commune
                                                                    "surf_lcz_group_com", # Surface commune couverte par un groupe LCZ
                                                                    "surf_id_filo", # Surface LCZ-FILO dd'une catégorie LCZ
                                                                    "label", # Nom de la commune
                                                                    'ind_w', 'men_w',
       'men_pauv_w', 'men_1ind_w', 'men_5ind_w', 'men_prop_w', 'men_fmp_w',
       'ind_snv_w', 'men_surf_w', 'men_coll_w', 'men_mais_w', 'log_av45_w',
       'log_45_70_w', 'log_70_90_w', 'log_ap90_w', 'log_inc_w', 'log_soc_w',
       'ind_0_3_w', 'ind_4_5_w', 'ind_6_10_w', 'ind_11_17_w', 'ind_18_24_w',
       'ind_25_39_w', 'ind_40_54_w', 'ind_55_64_w', 'ind_65_79_w', 'ind_80p_w',
       'ind_inc_w'])

for i in range(len(lcz_folders)):

    print("Traitement de la zone ", lcz_folders[i], "(", i+1, "/", len(lcz_folders), ")")

    # On charge le fichier
    files = os.listdir(os.path.join(download_dir, subfold_lcz,lcz_folders[i]))
    file = [f for f in files if f.endswith(".shp")][0]
    gdf = gpd.read_file(os.path.join(download_dir, subfold_lcz, lcz_folders[i], file))

    # On dissout pour obtenir le polygone de la zone
    gdf_zone = gdf.dissolve()
    # On fait une jointure spatiale avec le fichier des communes
    communes_lcz = communes.sjoin(gdf_zone, how="inner")
    # On récupère les codes officiels des communes qui sont dans la zone
    codes = communes_lcz.official_c.unique()


    # On calcule l'aire de chaque commune
    communes_lcz["surf_com_official"] = communes_lcz.area/10**6


    # On joint à la table des communes et on calcule la part de la commune dans la zone
    communes_in_lcz = communes[communes["official_c"].isin(codes)].merge(communes_lcz.filter(items=["official_c","surf_com_official"]), on="official_c", how="left")
    communes_in_lcz = communes_in_lcz.filter(items=["official_c","surf_com_official","geometry","label"])

    # On divise les zones LCZ par commune
    gdf_com = gdf.overlay(communes_in_lcz, how="intersection")
    # on calcule la surface de chaque polygone
    gdf_com["surf_lcz_id_com"] = gdf_com.area/10**6

    # On calcule la surface LCZ par commune
    gdf_surf_com = (gdf_com
                    .filter(items=["official_c","surf_lcz_id_com","lcz"])
                    .groupby(["official_c","lcz"]).sum().reset_index())

    gdf_surf_com = gdf_surf_com.rename(columns={"surf_lcz_id_com":"surf_lcz_group_com"})
    gdf_com = gdf_com.merge(gdf_surf_com, on=["official_c","lcz"],how="left")


    # On effectue la fusion avec les données FILOSOFI
    ## On commence par ne conserver que l'id du carreau, la superficie et la géometrie
    filo_geo = filosofi.filter(items=["geometry","idcar_200m"])

    # On joint les données FILOSOFI par un overlay
    filo_lcz = gpd.overlay(gdf_com.filter(
        items=["lcz","geometry","hre","bur","official_c","label","surf_lcz_id_com","surf_lcz_group_com","surf_com_official"]), 
        filo_geo, how="intersection")


    # H : On augmente la hauteur moyenne mini à 4m pour les zones résidentielles si la valeur est égale à 0 
    # (une valeur de 0 semble illogique et a également pour effet de supprimer la donnée lors de la pondération)
    filo_lcz.loc[
        (filo_lcz["hre"] == 0) & (filo_lcz["lcz"].isin(["1", "2", "3", "4", "5", "6", "7", "8", "9"])),
        "hre"
    ] = 4

    # On calcule la surface de chaque polygone
    filo_lcz["surf_id_filo"] = filo_lcz.geometry.area/10**6
    # On calcule le pseudo-volume de chaque polygone
    # H : On fait une hyothèse de hauteur de 3m pour les classes A à G (le minimum observé pour LCZ = 9) 
    # et de densité de 1% (soit 10% de moins que classe 9)// C'est une hypothèse
    filo_lcz["pseudovol_id_filo"] = (filo_lcz["surf_id_filo"]*filo_lcz["hre"]*filo_lcz["bur"] 
                                    * (1-filo_lcz["lcz"].isin(["A","B","C","D","E","F","G"])) 
                                    + filo_lcz["lcz"].isin(["A","B","C","D","E","F","G"])
                                    * filo_lcz["surf_id_filo"]*3*1)

    ## On présente le découpage de chaque carreau filo en LCZ
    filo_lcz_surf = (filo_lcz
                    .filter(items=["lcz","idcar_200m","surf_id_filo","pseudovol_id_filo","label","official_c"])
                    .groupby(["idcar_200m","lcz"])
                    .sum()
                    .reset_index()
                    .pivot_table(index="idcar_200m", 
                                columns="lcz", 
                                values=["surf_id_filo","pseudovol_id_filo"], 
                                fill_value=0)
                    .reset_index())
    filo_lcz_surf["surf_filo"] = sum([filo_lcz_surf[col] 
                                    for col in filo_lcz_surf.columns 
                                    if col[0] == "surf_id_filo"])
    filo_lcz_surf["pseudovol_filo"] = sum([filo_lcz_surf[col] 
                                        for col in filo_lcz_surf.columns 
                                        if col[0] == "pseudovol_id_filo"])



    ## On calcule la surface totale de chaque carreau
    filo_lcz_surf["surf_non_constructed"] = sum([filo_lcz_surf[col] for col in filo_lcz_surf.columns if col[0]== "surf_id_filo" and col[1] in ["A","B","C","D","E","F","G"]])
    filo_lcz_surf["pseudovol_non_constructed"] = sum([filo_lcz_surf[col] for col in filo_lcz_surf.columns if col[0]== "pseudovol_id_filo" and col[1] in ["A","B","C","D","E","F","G"]])
    filo_lcz_surf["filo_not_constructed"] = (filo_lcz_surf["surf_filo"] - filo_lcz_surf["surf_non_constructed"]) == 0
    filo_lcz_surf["filo_sub_flat"] = filo_lcz_surf["filo_not_constructed"] * filo_lcz_surf["surf_filo"] + (1 - filo_lcz_surf["filo_not_constructed"]) * (filo_lcz_surf["surf_filo"]- filo_lcz_surf["surf_non_constructed"])
    # H: On assigne une hauteur de 3m aux surfaces non construites et un taux d'occupation de 1%
    filo_lcz_surf["filo_sub_psdovol"] =  filo_lcz_surf["filo_not_constructed"] * filo_lcz_surf["pseudovol_filo"] + (1 - filo_lcz_surf["filo_not_constructed"]) * (filo_lcz_surf["pseudovol_filo"] - filo_lcz_surf["pseudovol_non_constructed"])
    ## On garde la table de conversion
    to_drop = ['surf_id_filo', 'pseudovol_id_filo']
    conversion_table =filo_lcz_surf.drop(columns=filo_lcz_surf.columns[filo_lcz_surf.columns.get_level_values(0).isin(to_drop)])
    conversion_table.columns = conversion_table.columns.droplevel(1)

    # On fusionnes les carreaux FILO/LCZ avec la table des poids de conversion des données FILO
    filo_lcz_w = filo_lcz.merge(conversion_table, on="idcar_200m", how="left")

    # On crée une varible binaire pour savoir si le carreau est compté 
    # H :(on compte les classes non baties lorsque pas de classes baties dans le carreau FILO)
    filo_lcz_w['is_accounted'] = (filo_lcz_w["lcz"].isin(["1","2","3","4","5","6","7","8","9"])) | filo_lcz_w["filo_not_constructed"]
    # On calcule le facteur de pondération pour chaque zone
    filo_lcz_w["flat_w_filo"]= (filo_lcz_w["surf_id_filo"]/filo_lcz_w["filo_sub_flat"])* filo_lcz_w["is_accounted"]
    filo_lcz_w["pseudovol_w_filo"]= (filo_lcz_w["pseudovol_id_filo"]/filo_lcz_w["filo_sub_psdovol"])* filo_lcz_w["is_accounted"]

    # On fusionne les carreaux et les données FILOSOFI
    filo_lcz_values = filo_lcz_w.merge(filosofi.drop(columns=["geometry",
                                        "idcar_1km",
                                        "idcar_nat",
                                        "i_est_200",
                                        "i_est_1km",
                                        "lcog_geo"]),
                    on="idcar_200m",
                    how="left")

    # On calcule les valeurs avec les poids "plats"
    filo_lcz_values_flat= weighted_filo(filo_lcz_values,"flat_w_filo",[
        'ind', 'men', 'men_pauv', 'men_1ind', 'men_5ind',
        'men_prop', 'men_fmp', 'ind_snv', 'men_surf', 'men_coll', 'men_mais',
        'log_av45', 'log_45_70', 'log_70_90', 'log_ap90', 'log_inc', 'log_soc',
        'ind_0_3', 'ind_4_5', 'ind_6_10', 'ind_11_17', 'ind_18_24', 'ind_25_39',
        'ind_40_54', 'ind_55_64', 'ind_65_79', 'ind_80p', 'ind_inc',
    ])
    filo_lcz_values_flat.to_file(os.path.join(prod_dir, "geo/lcz/filo_com",''.join([lcz_folders[i],"-flat-FILO.geojson"])), driver='GeoJSON')

    # Calcul du résumé des valeurs par LCZ par commune
    summary_lcz_filo_flat = filo_lcz_values_flat.filter(items=["lcz", # Code LCZ
                                                                    "official_c", # CODE INSEE
                                                                    "surf_com_official", # Surface globale commune
                                                                    "surf_lcz_group_com", # Surface commune couverte par un groupe LCZ
                                                                    "surf_id_filo", # Surface LCZ-FILO dd'une catégorie LCZ
                                                                    "label", # Nom de la commune
                                                                    'ind_w', 'men_w',
        'men_pauv_w', 'men_1ind_w', 'men_5ind_w', 'men_prop_w', 'men_fmp_w',
        'ind_snv_w', 'men_surf_w', 'men_coll_w', 'men_mais_w', 'log_av45_w',
        'log_45_70_w', 'log_70_90_w', 'log_ap90_w', 'log_inc_w', 'log_soc_w',
        'ind_0_3_w', 'ind_4_5_w', 'ind_6_10_w', 'ind_11_17_w', 'ind_18_24_w',
        'ind_25_39_w', 'ind_40_54_w', 'ind_55_64_w', 'ind_65_79_w', 'ind_80p_w',
        'ind_inc_w']).groupby(["lcz","official_c","label","surf_com_official","surf_lcz_group_com"]).sum().reset_index()

    summary_tot_lcz_filo_flat = pd.concat([summary_tot_lcz_filo_flat, summary_lcz_filo_flat])

    # On calcule les valeurs avec les poids "volumiques"
    filo_lcz_values_psdovol= weighted_filo(filo_lcz_values,"pseudovol_w_filo",[
        'ind', 'men', 'men_pauv', 'men_1ind', 'men_5ind',
        'men_prop', 'men_fmp', 'ind_snv', 'men_surf', 'men_coll', 'men_mais',
        'log_av45', 'log_45_70', 'log_70_90', 'log_ap90', 'log_inc', 'log_soc',
        'ind_0_3', 'ind_4_5', 'ind_6_10', 'ind_11_17', 'ind_18_24', 'ind_25_39',
        'ind_40_54', 'ind_55_64', 'ind_65_79', 'ind_80p', 'ind_inc',
    ])
    filo_lcz_values_psdovol.to_file(os.path.join(prod_dir, "geo/lcz/filo_com",''.join([lcz_folders[i],"-psdovol-FILO.geojson"])), driver='GeoJSON')

    # Calcul du résumé des valeurs par LCZ par commune
    summary_lcz_filo_psdovol = filo_lcz_values_psdovol.filter(items=["lcz", # Code LCZ
                                                                    "official_c", # CODE INSEE
                                                                    "surf_com_official", # Surface globale commune
                                                                    "surf_lcz_group_com", # Surface commune couverte par un groupe LCZ
                                                                    "surf_id_filo", # Surface LCZ-FILO dd'une catégorie LCZ
                                                                    "label", # Nom de la commune
                                                                    'ind_w', 'men_w',
        'men_pauv_w', 'men_1ind_w', 'men_5ind_w', 'men_prop_w', 'men_fmp_w',
        'ind_snv_w', 'men_surf_w', 'men_coll_w', 'men_mais_w', 'log_av45_w',
        'log_45_70_w', 'log_70_90_w', 'log_ap90_w', 'log_inc_w', 'log_soc_w',
        'ind_0_3_w', 'ind_4_5_w', 'ind_6_10_w', 'ind_11_17_w', 'ind_18_24_w',
        'ind_25_39_w', 'ind_40_54_w', 'ind_55_64_w', 'ind_65_79_w', 'ind_80p_w',
        'ind_inc_w']).groupby(["lcz","official_c","label","surf_com_official","surf_lcz_group_com"]).sum().reset_index()

    summary_tot_lcz_filo_psdovol = pd.concat([summary_tot_lcz_filo_psdovol, summary_lcz_filo_psdovol])

# Sauvegarder des résultats globaux
summary_tot_lcz_filo_psdovol.to_csv(os.path.join(prod_dir, "geo/lcz",'summary_tot_lcz_filo_psdovol.csv'), index=False)
summary_tot_lcz_filo_flat.to_csv(os.path.join(prod_dir, "geo/lcz",'summary_tot_lcz_filo_flat.csv'), index=False)

Traitement de la zone  lcz-spot-2022-le-havre ( 1 / 83 )


/var/folders/pl/xywgn7qd4sv_s094vn5h3dtr0000gn/T/ipykernel_91391/1438938172.py:173: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  summary_tot_lcz_filo_flat = pd.concat([summary_tot_lcz_filo_flat, summary_lcz_filo_flat])
/var/folders/pl/xywgn7qd4sv_s094vn5h3dtr0000gn/T/ipykernel_91391/1438938172.py:200: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  summary_tot_lcz_filo_psdovol = pd.concat([summary_tot_lcz_filo_psdovol, summary_lcz_filo_psdovol])


Traitement de la zone  lcz-spot-2022-quimper ( 2 / 83 )


/var/folders/pl/xywgn7qd4sv_s094vn5h3dtr0000gn/T/ipykernel_91391/1438938172.py:74: UserWarning: `keep_geom_type=True` in overlay resulted in 3 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  filo_lcz = gpd.overlay(gdf_com.filter(


Traitement de la zone  lcz-spot-2022-grenoble ( 3 / 83 )


...

/var/folders/pl/xywgn7qd4sv_s094vn5h3dtr0000gn/T/ipykernel_91391/1438938172.py:74: UserWarning: `keep_geom_type=True` in overlay resulted in 9 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  filo_lcz = gpd.overlay(gdf_com.filter(


Traitement de la zone  lcz-spot-2022-frejus ( 82 / 83 )


/var/folders/pl/xywgn7qd4sv_s094vn5h3dtr0000gn/T/ipykernel_91391/1438938172.py:74: UserWarning: `keep_geom_type=True` in overlay resulted in 8 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  filo_lcz = gpd.overlay(gdf_com.filter(


Traitement de la zone  lcz-spot-2022-marseille ( 83 / 83 )


/var/folders/pl/xywgn7qd4sv_s094vn5h3dtr0000gn/T/ipykernel_91391/1438938172.py:74: UserWarning: `keep_geom_type=True` in overlay resulted in 44 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  filo_lcz = gpd.overlay(gdf_com.filter(
